In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

Open driver e get url of website

In [2]:
URL = "https://www.autotrader.co.uk/cars"

driver = webdriver.Chrome()
driver.get(URL)

In [3]:
#Reject cookies
driver.implicitly_wait(10)
frame_0 = driver.find_element(By.CSS_SELECTOR, "iframe[src='https://cmpv2.autotrader.co.uk/index.html?message_id=1371540&consentUUID=undefined&preload_message=true&hasCsp=true&version=v1&consent_origin=https%3A%2F%2Fcmpv2.autotrader.co.uk%2Fconsent%2Ftcfv2&mms_origin=https%3A%2F%2Fcmpv2.autotrader.co.uk%2Fmms%2Fv2']")
driver.switch_to.frame(frame_0)
No_cookies = driver.find_element(By.XPATH, "//button[contains(text(), 'Reject All')]")
No_cookies.click()

In [4]:
driver.switch_to.default_content()

Scrap the form in website to first get all the make (brands) and then for each make get all the models

In [5]:
make_select = Select(WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.ID, "make"))
))
makes = [(opt.get_attribute("value"))
         for opt in make_select.options
         if opt.text.strip()]

In [6]:
rows = []

In [8]:
for val in makes:
    make_select = Select(WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.ID, "make"))
))
    make_select.select_by_value(val)
    time.sleep(1.2)

    model_select = Select(WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.ID, "model"))
))
    models = [o.get_attribute("value") for o in model_select.options
              if o.text.strip() and o.text.strip().lower() not in {"any","todas","todos","selecionar"}]

    for md in models:
        rows.append({"make": val, "model": md})

driver.quit()

In [9]:
rows

[{'make': 'Abarth', 'model': '124 Spider'},
 {'make': 'Abarth', 'model': '500'},
 {'make': 'Abarth', 'model': '500C'},
 {'make': 'Abarth', 'model': '500e'},
 {'make': 'Abarth', 'model': '500e C'},
 {'make': 'Abarth', 'model': '595'},
 {'make': 'Abarth', 'model': '595C'},
 {'make': 'Abarth', 'model': '600e'},
 {'make': 'Abarth', 'model': '695'},
 {'make': 'Abarth', 'model': '695C'},
 {'make': 'Abarth', 'model': 'Grande Punto'},
 {'make': 'Abarth', 'model': 'Punto Evo'},
 {'make': 'AC', 'model': 'Cobra'},
 {'make': 'AC', 'model': 'Dax Cobra'},
 {'make': 'Aixam', 'model': 'Coupe'},
 {'make': 'AK', 'model': 'Cobra'},
 {'make': 'Alfa Romeo', 'model': '145'},
 {'make': 'Alfa Romeo', 'model': '147'},
 {'make': 'Alfa Romeo', 'model': '156'},
 {'make': 'Alfa Romeo', 'model': '159'},
 {'make': 'Alfa Romeo', 'model': '159 Sportwagon'},
 {'make': 'Alfa Romeo', 'model': '164'},
 {'make': 'Alfa Romeo', 'model': '2000'},
 {'make': 'Alfa Romeo', 'model': '4C'},
 {'make': 'Alfa Romeo', 'model': '75'},


In [10]:
#Create a dataframe with all the pairs make-model
df_ref = pd.DataFrame(rows).drop_duplicates()

Map some specific values to match train.csv dataset

In [11]:
brand_map = {
    "Vauxhall": "Opel",
    "Mercedes-Benz": "Mercedes",
    "Volkswagen":"VW"
}

df_ref["make"] = df_ref["make"].map(brand_map).fillna(df_ref["make"])

In [12]:
new_rows_data = [
    {'make': 'Opel', 'model': 'gtc'},
    {'make': 'VW', 'model': 'shuttle'},
    {'make': 'Audi', 'model': 'a6'},
    {'make': 'Skoda', 'model': 'yeti outdoor'}
]

df_ref = pd.concat([df_ref, pd.DataFrame(new_rows_data)], ignore_index=True)

In [13]:
is_mercedes = df_ref["make"].eq("Mercedes")
has_class   = df_ref["model"].str.contains(r"\bclass\b", case=False)

In [14]:
to_dup = df_ref[is_mercedes & ~has_class].copy()
to_dup["model"] = to_dup["model"] + " Class"

In [16]:
df_final = (
    pd.concat([df_ref, to_dup], ignore_index=True)
      .drop_duplicates(subset=["make","model"])
      .sort_values(["make","model"])
      .reset_index(drop=True)
)

In [17]:
df_final

,make,model
0,AC,Cobra
1,AC,Dax Cobra
2,AK,Cobra
3,Abarth,124 Spider
4,Abarth,500
...,...,...
1605,Westfield,Sei
1606,Westfield,Sport
1607,Westfield,Xi
1608,XPENG,G6


In [18]:
#export as csv
df_final.to_csv("make_model_catalog.csv", index=False)